<a href="https://colab.research.google.com/github/JacopoMangiavacchi/animegan2-coreml/blob/master/convert_coreml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install coremltools

In [2]:
import os
import torch
import coremltools

In [3]:
!git clone https://github.com/jacopomangiavacchi/animegan2-coreml

Cloning into 'animegan2-coreml'...
remote: Enumerating objects: 79, done.
remote: Counting objects: 100% (79/79), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 79 (delta 31), reused 36 (delta 6), pack-reused 0
Unpacking objects: 100% (79/79), done.


In [4]:
os.chdir(f'./animegan2-coreml')

In [5]:
from model import Generator

device = 'cpu'
torch.set_grad_enabled(False)

model = Generator().eval().to(device)

In [6]:
ckpt = torch.load(f"weights/celeba_distill.pt", map_location=device)
# ckpt = torch.load(f"weights/face_paint_512_v2.pt", map_location=device)
# ckpt = torch.load(f"weights/paprika.pt", map_location=device)
model.load_state_dict(ckpt)

<All keys matched successfully>

In [7]:
image_sizes = [256, 512, 1024]

for image_size in image_sizes:
    image = torch.randn(1, 3, image_size, image_size)
    output = model(image.to(device))

    # scripted_model = torch.jit.script(model)
    traced = torch.jit.trace(model, image)

    image_input = coremltools.ImageType(name="input_1", shape=(1, 3, image_size, image_size))

    mlmodel = coremltools.converters.convert(
      traced,
      inputs=[image_input],
      minimum_deployment_target=coremltools.target.iOS15,
      compute_units=coremltools.ComputeUnit.CPU_AND_GPU
    )

    mlmodel.save(f'animegan2-celeba-distil-{image_size}.mlpackage')    
    spec = coremltools.utils.load_spec(f"animegan2-celeba-distil-{image_size}.mlpackage")

    output = spec.description.output[0]

    output.type.imageType.colorSpace = coremltools.proto.FeatureTypes_pb2.ImageFeatureType.RGB
    output.type.imageType.height = image_size
    output.type.imageType.width = image_size

    coremltools.utils.save_spec(spec, f"animegan2-celeba-distil-{image_size}.mlpackage")

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2359: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  _verify_batch_size([input.size(0) * input.size(1) // num_groups, num_groups] + list(input.size()[2:]))
Running MIL Common passes:   6%|▌         | 2/34 [00:00<00:01, 18.30 passes/s]/usr/local/lib/python3.7/dist-packages/coremltools/converters/mil/mil/passes/name_sanitization_utils.py:129: UserWarning: Output, '457', of the source model, has been renamed to 'var_457' in the Core ML model.
  warnings.warn(msg.format(var.name, new_name))
Running MIL Clean up passes: 100%|██████████| 9/9 [00:08<00:00,  1.04 passes/s]


In [8]:
!zip -r /content/animegan2-coreml/animegan2-celeba-distil-1024.zip /content/animegan2-coreml/animegan2-celeba-distil-1024.mlpackage
!zip -r /content/animegan2-coreml/animegan2-celeba-distil-512.zip /content/animegan2-coreml/animegan2-celeba-distil-512.mlpackage
!zip -r /content/animegan2-coreml/animegan2-celeba-distil-256.zip /content/animegan2-coreml/animegan2-celeba-distil-256.mlpackage

  adding: content/animegan2-coreml/animegan2-celeba-distil-1024.mlpackage/ (stored 0%)
  adding: content/animegan2-coreml/animegan2-celeba-distil-1024.mlpackage/Manifest.json (deflated 59%)
  adding: content/animegan2-coreml/animegan2-celeba-distil-1024.mlpackage/Data/ (stored 0%)
  adding: content/animegan2-coreml/animegan2-celeba-distil-1024.mlpackage/Data/com.apple.CoreML/ (stored 0%)
  adding: content/animegan2-coreml/animegan2-celeba-distil-1024.mlpackage/Data/com.apple.CoreML/animegan2-celeba-distil-1024.mlmodel (deflated 88%)
  adding: content/animegan2-coreml/animegan2-celeba-distil-1024.mlpackage/Data/com.apple.CoreML/weights/ (stored 0%)
  adding: content/animegan2-coreml/animegan2-celeba-distil-1024.mlpackage/Data/com.apple.CoreML/weights/weight.bin (deflated 8%)
  adding: content/animegan2-coreml/animegan2-celeba-distil-512.mlpackage/ (stored 0%)
  adding: content/animegan2-coreml/animegan2-celeba-distil-512.mlpackage/Manifest.json (deflated 59%)
  adding: content/animegan2

In [9]:
os.chdir(f'..')